In [0]:
!pip3 install ivis

     |████████████████████████████████| 645kB 7.6MB/s 
  Created wheel for annoy: filename=annoy-1.16.2-cp36-cp36m-linux_x86_64.whl size=310416 sha256=c1a4eaea4664b1bb8dd3d4b259b779e8df5ad754ac0eddf255d600816eb5b4ff
  Stored in directory: /root/.cache/pip/wheels/d3/d7/68/3795670ef4c6781fc10df0d6cf83b922244aa28cd9489d1176
Successfully built annoy


In [7]:
from ivis import Ivis
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

from keras.layers import BatchNormalization, Dense, Activation, Lambda, Input
from keras.models import Sequential, Model
from keras.callbacks import LearningRateScheduler
from keras import backend as K

from imblearn.over_sampling import SMOTE, RandomOverSampler

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import GaussianNB

import itertools
import time

ModuleNotFoundError: No module named 'ivis'

In [0]:
base_path = '.'

In [0]:
data_fname = 'ant.csv'

In [0]:
df = pd.read_csv(f'{base_path}/{data_fname}')

In [0]:
x = df.drop('bug', axis=1)
y = df['bug']

In [0]:
batch_size = 128
num_classes = 2

# Helper functions

In [0]:
def get_model(units=2, n_layers=1, deep_learner=False, batchnorm=False):
    """
    Returns a model.
    
    Params:
    =======
    units: int - Number of input units. Only for deep learners.
    layers: int - Number of hidden layers. Only for deep learners.
    deep_learner: boolean - To use deep learner, set to True.
    batchnorm: boolean - If True, uses Batch Normalization.
    
    Returns:
    ========
    An instance of keras.Sequential or AdaBoost from sklearn.
    """
    if deep_learner:
        layers = []
        
        # Add first hidden layer set.
        first = Dense(units, input_shape=(units,))
        layers.append(first)
        
        if batchnorm:
            layers.append(BatchNormalization())
        
        layers.append(Activation('relu'))
        
        # Add the other hidden layers.
        for i in range(n_layers - 1):
            layers.append(Dense(units))
            
            if batchnorm:
                layers.append(BatchNormalization())
            
            layers.append(Activation('relu'))
        
        # Add the output layer.
        layers.append(Dense(1))
        layers.append(Activation('sigmoid'))
        
        return Sequential(layers)
    else:
        return AdaBoostClassifier()

In [0]:
def get_oversampler(name=None):
    """
    Returns an oversampler.
    
    Params:
    =======
    name: str or None - One of None, 'smote', and 'random'.
    """
    if name is None:
        return None
    if name == 'smote':
        return SMOTE()
    if name == 'random':
        return RandomOverSampler()

## Functions to run experiments

In [0]:
def run_one_experiment(x_resampled, y_resampled, x_test, y_test, deep_learner=False, n_layers=1, batchnorm=False, lipschitz_lr=False, 
                       reduce_data=False, embedding_dims=2, k=7, oversample=False, oversampler=None):
    """
    Runs one experiment.
    
    Params:
    =======
    x, y: np.array - Input data
    deep_learner: boolean - If True, uses a deep learner.
    layers: int - Number of hidden layers. Deep learners only.
    batchnorm: boolean - If True, uses Batch Normalization in deep learner.
    lipschitz_lr: boolean - If True, uses LipschitzLR
    reduce_data: boolean - If True, reduces data using Ivis.
    k: int - k used in Ivis reduction.
    embedding_dims: int - Number of dimensions to reduce data to
    oversample: boolean - If True, oversamples dataset.
    oversampler: str or None - One of None, 'smote', and 'random'.
    
    Returns:
    ========
    prec, recall, f1, time - Precision, recall, and F-1 scores of the model, along with runtime
    """    
    
    # Step 4: Get the model
    model = get_model(units=x_resampled.shape[1],n_layers=n_layers, deep_learner=deep_learner, batchnorm=batchnorm)
    
    # Step 5: Train model
    if deep_learner:
        ####################
        # LipschitzLR code #
        ####################    
        def lr_schedule(epoch):
            """Learning Rate Schedule
            # Arguments
                epoch (int): The number of epochs
            # Returns
                lr (float32): learning rate
            """

            Kz = 0.
            for i in range((len(x_resampled) - 1) // batch_size + 1):
                start_i = i * batch_size
                end_i = start_i + batch_size
                xb = x_resampled[start_i:end_i]

                activ = np.linalg.norm(func([xb]))
                if activ > Kz:
                    Kz = activ

            K_ = ((num_classes - 1) * Kz) / (num_classes * batch_size)
            lr = 1 / K_
            #print('Epoch', epoch + 1, 'LR =', lr)
            return lr
        ####################
        
        if lipschitz_lr:
            model.compile(loss='binary_crossentropy',
                  optimizer='sgd')
            func = K.function([model.layers[0].input], [model.layers[-2].output])
            lr_scheduler = LearningRateScheduler(lr_schedule)
            
            start = time.time()
            model.fit(x_resampled, y_resampled, batch_size=batch_size, verbose=0, 
                      validation_data=(x_test, y_test), epochs=100, callbacks=[lr_scheduler])
            end = time.time()
        else:
            model.compile(loss='binary_crossentropy',
                  optimizer='adam')
            
            start = time.time()
            model.fit(x_resampled, y_resampled, batch_size=batch_size, verbose=0, 
                      validation_data=(x_test, y_test), epochs=100)
            end = time.time()
        
    else:
        start = time.time()
        model.fit(x_resampled, y_resampled)
        end = time.time()
    
    # Step 6: Evaluate model
    evaluators = [precision_score, recall_score, f1_score]
    if deep_learner:
        prec, recall, f1 = [f(y_test, model.predict_classes(x_test)) for f in evaluators]
    else:
        prec, recall, f1 = [f(y_test, model.predict(x_test)) for f in evaluators]
    
    return prec, recall, f1, (end - start)

In [0]:
def run_20_experiments(reduce_data=False, oversample=False, oversampler=None, 
                       embedding_dims=2, k=7, lipschitz_lr=False, **kwargs):
    global x, y
    prec = []
    recall = []
    f1 = []
    times = []
    
    count = 0

    # Step 1: Reduce dimensions --> (x_reduced, y)
    x_reduced = np.array(x)
    y = np.array(y)
    if reduce_data:
        ivis = Ivis(embedding_dims=embedding_dims, k=k, verbose=0)
        x_reduced = ivis.fit_transform(np.array(x))
    
    # Step 2: Split data --> (x_train, y_train)
    x_train, x_test, y_train, y_test = train_test_split(x_reduced, y)
    
    # Step 3: Oversample data --> (x_resampled, y_resampled)
    x_resampled, y_resampled = x_train, y_train
    if oversample:
        sampler = get_oversampler(name=oversampler)
        x_resampled, y_resampled = sampler.fit_resample(x_train, y_train)

    while count != 20:
        try:          
            precision, rec, f1_, time_ = run_one_experiment(x_resampled, y_resampled,
                                                            x_test, y_test, lipschitz_lr=lipschitz_lr, **kwargs)
            prec.append(precision)
            recall.append(rec)
            f1.append(f1_)
            times.append(time_)
            count += 1
        except KeyboardInterrupt:
            raise
        except:
            print('Retrying...')
            if lipschitz_lr:
                pass
            else:
                raise
    
    print('Precision:', prec, '\nRecall:', recall, '\nF1:', f1, '\nRuntime:', times)

# Experiments

In [8]:
import itertools

In [11]:
def get_experimental_config():
    dl_options = [False, True]
    layers_options = [1, 2, 3]
    batchnorm_options = [True, False]  # Reduces #experiments
    lipschitz_options = [False, True]
    reduce_options = [False, True]
    embedding_options = [2, 3]
    k_options = [3, 4, 5, 6, 7]
    oversample_options = [False, True]
    oversampler_options = ['random', None]  # Not using smote
    activation_options = ['relu', 'linear']
    
    for (dl, layers, bn, lipschitz, reduce, emb, k, over, sampler, activation) in itertools.product(dl_options,
                                                                                       layers_options,
                                                                                       batchnorm_options,
                                                                                       lipschitz_options,
                                                                                       reduce_options,
                                                                                       embedding_options,
                                                                                       k_options,
                                                                                       oversample_options,
                                                                                       oversampler_options,
                                                                                       activation_options):
        # Constraints:
        if dl == False:
            if layers != 1 or lipschitz != False or activation != 'relu':
                continue
        if not reduce:
            if emb != 2 or k != 3:
                continue
        if not over:
            if sampler is not None:
                continue
        else:
            if sampler is None:
                continue
        
        # Skip over completed configs
        config = {
            'deep_learner': dl,
            'n_layers': layers,
            'batchnorm': bn,
            'lipschitz_lr': lipschitz,
            'reduce_data': reduce,
            'embedding_dims': emb,
            'k': k,
            'oversample': over,
            'oversampler': sampler,
            'activation': activation
        }
        complete = []
        if config in complete:
            continue
        
        yield config

In [5]:
len(x)

NameError: name 'x' is not defined

In [12]:
# Count number of experiments
print(len(list(get_experimental_config())) * 20)

11440


# Experiments

In [0]:
for i, config in enumerate(get_experimental_config()):
    print('Experiment', i + 1, '\b:', config)
    run_20_experiments(**config)
    print()

Experiment 1 : {'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': False, 'oversampler': None}
Precision: [0.6296296296296297, 0.6, 0.8333333333333334, 0.6666666666666666, 0.7741935483870968, 0.631578947368421, 0.696969696969697, 0.5769230769230769, 0.64, 0.6521739130434783, 0.7692307692307693, 0.6521739130434783, 0.5333333333333333, 0.7272727272727273, 0.5263157894736842, 0.7, 0.7407407407407407, 0.6923076923076923, 0.5909090909090909, 0.5185185185185185] 
Recall: [0.4857142857142857, 0.40540540540540543, 0.46875, 0.42857142857142855, 0.5333333333333333, 0.3333333333333333, 0.5348837209302325, 0.4166666666666667, 0.42105263157894735, 0.38461538461538464, 0.45454545454545453, 0.3488372093023256, 0.5333333333333333, 0.36363636363636365, 0.23255813953488372, 0.3333333333333333, 0.43478260869565216, 0.4090909090909091, 0.28888888888888886, 0.3684210526315789] 
F1: [0.5483870967741936, 0.483870967

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: P

Precision: [0.0, 0.0, 0.6666666666666666, 0.0, 0.5405405405405406, 0.72, 0.0, 0.6666666666666666, 0.7368421052631579, 0.0, 0.24242424242424243, 0.5714285714285714, 0.75, 0.0, 0.0, 0.75, 0.0, 0.8235294117647058, 0.0, 0.6875] 
Recall: [0.0, 0.0, 0.05128205128205128, 0.0, 0.45454545454545453, 0.3673469387755102, 0.0, 0.05, 0.42424242424242425, 0.0, 0.2222222222222222, 0.2857142857142857, 0.36585365853658536, 0.0, 0.0, 0.20930232558139536, 0.0, 0.3181818181818182, 0.0, 0.2619047619047619] 
F1: [0.0, 0.0, 0.09523809523809523, 0.0, 0.49382716049382713, 0.4864864864864865, 0.0, 0.09302325581395349, 0.5384615384615385, 0.0, 0.2318840579710145, 0.38095238095238093, 0.4918032786885247, 0.0, 0.0, 0.32727272727272727, 0.0, 0.45901639344262296, 0.0, 0.37931034482758624] 
Runtime: [17.786104440689087, 18.17351746559143, 18.566311597824097, 18.785359859466553, 19.23588752746582, 19.82448959350586, 20.096041202545166, 20.508277654647827, 20.730725288391113, 21.31090807914734, 21.58726739883423, 22.014

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: P

Precision: [1.0, 0.3333333333333333, 0.0, 0.0, 0.6666666666666666, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.29411764705882354, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.45714285714285713, 0.72] 
Recall: [0.05555555555555555, 0.029411764705882353, 0.0, 0.0, 0.21621621621621623, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.11904761904761904, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.34615384615384615] 
F1: [0.10526315789473684, 0.05405405405405406, 0.0, 0.0, 0.326530612244898, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.16949152542372883, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4776119402985075, 0.4675324675324676] 
Runtime: [31.142149448394775, 31.70679783821106, 32.15516543388367, 32.14604687690735, 32.55091834068298, 32.87303423881531, 33.156577587127686, 33.683571338653564, 33.88092517852783, 34.26084876060486, 34.81228470802307, 36.35526943206787, 35.32514977455139, 35.58494257926941, 36.16671848297119, 36.38939046859741, 36.92014122009277, 37.13206076622009, 37.560842990875244, 37.79997134208679]

Experiment 6 : {'deep_learner': True, '

...

Precision: [0.43548387096774194, 0.3333333333333333, 0.3950617283950617, 0.30337078651685395, 0.38095238095238093, 0.37777777777777777, 0.4032258064516129, 0.4375, 0.2, 0.43478260869565216, 0.3728813559322034, 0.5066666666666667, 0.4, 0.32142857142857145, 0.5128205128205128, 0.4625, 0.3333333333333333, 0.16666666666666666, 0.44155844155844154, 0.3380281690140845] 
Recall: [0.7105263157894737, 0.7105263157894737, 0.7619047619047619, 0.6923076923076923, 0.8, 0.7906976744186046, 0.7352941176470589, 0.7777777777777778, 0.14, 0.851063829787234, 0.6470588235294118, 0.6909090909090909, 0.8372093023255814, 0.84375, 0.8163265306122449, 0.8043478260869565, 0.71875, 0.13043478260869565, 0.8095238095238095, 0.6666666666666666] 
F1: [0.54, 0.453781512605042, 0.5203252032520325, 0.421875, 0.5161290322580645, 0.5112781954887218, 0.5208333333333333, 0.56, 0.1647058823529412, 0.5755395683453237, 0.4731182795698925, 0.5846153846153845, 0.5413533834586467, 0.4655172413793104, 0.6299212598425196, 0.587301

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: P

Precision: [0.09090909090909091, 0.3333333333333333, 0.5, 0.68, 0.0, 0.7, 0.4, 0.0, 0.7391304347826086, 0.4482758620689655, 0.75, 0.5882352941176471, 0.5185185185185185, 0.43333333333333335, 0.8, 0.5882352941176471, 0.0, 0.0, 0.0, 0.0] 
Recall: [0.02564102564102564, 0.025, 0.30952380952380953, 0.3617021276595745, 0.0, 0.16279069767441862, 0.17777777777777778, 0.0, 0.4358974358974359, 0.6842105263157895, 0.35294117647058826, 0.45454545454545453, 0.3684210526315789, 0.3611111111111111, 0.10256410256410256, 0.2222222222222222, 0.0, 0.0, 0.0, 0.0] 
F1: [0.04, 0.046511627906976744, 0.3823529411764706, 0.4722222222222222, 0.0, 0.2641509433962264, 0.24615384615384614, 0.0, 0.5483870967741935, 0.5416666666666666, 0.48, 0.5128205128205129, 0.4307692307692308, 0.39393939393939387, 0.18181818181818182, 0.3225806451612903, 0.0, 0.0, 0.0, 0.0] 
Runtime: [75.94429779052734, 74.8167188167572, 75.5016610622406, 77.31247305870056, 76.02598071098328, 76.33459997177124, 77.07845139503479, 77.370179653167

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


KeyboardInterrupt: ignored

In [0]:
for i, config in enumerate(get_experimental_config()):
    print('Experiment', i + 12, '\b:', config)
    run_20_experiments(**config)
    print()

Experiment 12 : {'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 2, 'k': 7, 'oversample': True, 'oversampler': 'random'}
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where














Precision: [0.47058823529411764, 0.625, 0.47692307692307695, 0.2196969696969697, 0.41333333333333333, 0.65, 0.4155844155844156, 0.4492753623188406, 0.4246575342465753, 0.44776119402985076, 0.4050632911392405, 0.40789473684210525, 0.4266666666666667, 0.5185185185185185, 0.4, 0.15151515151515152, 0.4411764705882353, 0.22297297297297297, 0.42105263157894735, 0.4189189189189189] 
Recall: [0.631578947368421, 0.5263157894736842, 0.8157894736842105, 0.7631578947368421, 0.8157894736842105, 0.34210526315789475, 0.8421052631578947, 0.8157894736842105, 0.8157894736842105, 0.7894736842105263, 0.8421052631578947, 0.815789473

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: P

Precision: [0.6470588235294118, 0.5714285714285714, 0.0, 0.0, 0.5263157894736842, 0.6756756756756757, 0.6666666666666666, 0.6551724137931034, 0.0, 0.0, 0.6428571428571429, 0.0, 0.0, 0.0, 0.6363636363636364, 1.0, 0.0, 0.3333333333333333, 0.6111111111111112, 0.5217391304347826] 
Recall: [0.2619047619047619, 0.2857142857142857, 0.0, 0.0, 0.23809523809523808, 0.5952380952380952, 0.5714285714285714, 0.4523809523809524, 0.0, 0.0, 0.42857142857142855, 0.0, 0.0, 0.0, 0.3333333333333333, 0.023809523809523808, 0.0, 0.047619047619047616, 0.2619047619047619, 0.2857142857142857] 
F1: [0.37288135593220334, 0.38095238095238093, 0.0, 0.0, 0.3278688524590164, 0.6329113924050633, 0.6153846153846153, 0.5352112676056338, 0.0, 0.0, 0.5142857142857143, 0.0, 0.0, 0.0, 0.43749999999999994, 0.046511627906976744, 0.0, 0.08333333333333333, 0.36666666666666675, 0.3692307692307692] 
Runtime: [6.476360559463501, 6.49149227142334, 6.842185974121094, 7.064222097396851, 7.395125389099121, 7.238799571990967, 7.50640082

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: P

Precision: [1.0, 1.0, 0.0, 0.0, 0.6521739130434783, 0.5714285714285714, 0.7, 0.6538461538461539, 0.0, 0.6666666666666666, 0.5, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.46153846153846156, 0.0, 0.6296296296296297] 
Recall: [0.041666666666666664, 0.020833333333333332, 0.0, 0.0, 0.3125, 0.4166666666666667, 0.14583333333333334, 0.3541666666666667, 0.0, 0.3333333333333333, 0.08333333333333333, 0.020833333333333332, 0.0, 0.0, 0.0, 0.0, 0.0, 0.125, 0.0, 0.3541666666666667] 
F1: [0.07999999999999999, 0.04081632653061225, 0.0, 0.0, 0.4225352112676057, 0.48192771084337344, 0.2413793103448276, 0.4594594594594595, 0.0, 0.4444444444444444, 0.14285714285714285, 0.04081632653061225, 0.0, 0.0, 0.0, 0.0, 0.0, 0.19672131147540983, 0.0, 0.45333333333333337] 
Runtime: [12.633641242980957, 12.94472622871399, 13.050775527954102, 13.304675102233887, 13.432825088500977, 13.594762086868286, 13.897267580032349, 13.797012329101562, 14.0407555103302, 14.19670557975769, 14.548513889312744, 14.568299531936646, 14.70547103881

...

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: [0.4507042253521127, 0.5185185185185185, 0.49122807017543857, 0.44, 0.46153846153846156, 0.453125, 0.48333333333333334, 0.46774193548387094, 0.4230769230769231, 0.4461538461538462, 0.43661971830985913, 0.47619047619047616, 0.4647887323943662, 0.4745762711864407, 0.4696969696969697, 0.47058823529411764, 0.4696969696969697, 0.47058823529411764, 0.46774193548387094, 0.5185185185185185] 
Recall: [0.8205128205128205, 0.717948717948718, 0.717948717948718, 0.8461538461538461, 0.7692307692307693, 0.7435897435897436, 0.7435897435897436, 0.7435897435897436, 0.8461538461538461, 0.7435897435897436, 0.7948717948717948, 0.7692307692307693, 0.8461538461538461, 0.717948717948718, 0.7948717948717948, 0.8205128205128205, 0.7948717948717948, 0.6153846153846154, 0.7435897435897436, 0.717948717948718] 
F1: [0.5818181818181818, 0.6021505376344085, 0.5833333333333334, 0.5789473684210525, 0.576923076923077, 0.5631067961165049, 0.5858585858585859, 0.5742574257425742, 0.5641025641025641, 0.5576923076

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: P

Precision: [0.6818181818181818, 0.5555555555555556, 0.0, 0.0, 0.6, 0.0, 0.0, 0.6875, 0.08333333333333333, 0.6923076923076923, 1.0, 0.0, 0.6666666666666666, 0.25, 0.8, 0.5882352941176471, 0.0, 0.0, 0.0, 0.0] 
Recall: [0.3125, 0.10416666666666667, 0.0, 0.0, 0.375, 0.0, 0.0, 0.22916666666666666, 0.041666666666666664, 0.375, 0.041666666666666664, 0.0, 0.4583333333333333, 0.020833333333333332, 0.25, 0.4166666666666667, 0.0, 0.0, 0.0, 0.0] 
F1: [0.42857142857142855, 0.1754385964912281, 0.0, 0.0, 0.4615384615384615, 0.0, 0.0, 0.34375, 0.05555555555555555, 0.48648648648648646, 0.07999999999999999, 0.0, 0.5432098765432098, 0.038461538461538464, 0.38095238095238093, 0.48780487804878053, 0.0, 0.0, 0.0, 0.0] 
Runtime: [24.59797239303589, 24.688132524490356, 24.91222047805786, 24.924108505249023, 25.111258029937744, 25.2746160030365, 25.604621648788452, 25.59337091445923, 25.298691749572754, 25.552463054656982, 25.876468658447266, 26.008085250854492, 26.05584716796875, 26.258033990859985, 26.992994

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
for i, config in enumerate(get_experimental_config()):
    print('Experiment', i + 19, '\b:', config)
    run_20_experiments(**config)
    print()

Experiment 19 : {'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 4, 'oversample': False, 'oversampler': None}
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
















/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: P

Precision: [0.6923076923076923, 0.0, 0.6363636363636364, 0.4444444444444444, 0.5217391304347826, 1.0, 0.5, 0.0, 1.0, 0.5263157894736842, 0.575, 0.5813953488372093, 0.5384615384615384, 0.0, 0.0, 0.4, 0.0, 0.5, 0.75, 0.0] 
Recall: [0.1956521739130435, 0.0, 0.15217391304347827, 0.2608695652173913, 0.5217391304347826, 0.043478260869565216, 0.1956521739130435, 0.0, 0.021739130434782608, 0.21739130434782608, 0.5, 0.5434782608695652, 0.15217391304347827, 0.0, 0.0, 0.5217391304347826, 0.0, 0.34782608695652173, 0.06521739130434782, 0.0] 
F1: [0.3050847457627119, 0.0, 0.2456140350877193, 0.32876712328767116, 0.5217391304347826, 0.08333333333333333, 0.28125, 0.0, 0.042553191489361694, 0.3076923076923077, 0.5348837209302325, 0.5617977528089888, 0.23728813559322035, 0.0, 0.0, 0.4528301886792453, 0.0, 0.41025641025641024, 0.12, 0.0] 
Runtime: [4.048007249832153, 4.045135736465454, 4.117496728897095, 4.15005350112915, 4.176778554916382, 4.318059921264648, 4.300934314727783, 4.511841058731079, 4.61923

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: [0.4, 0.425, 0.41333333333333333, 0.43478260869565216, 0.4507042253521127, 0.4, 0.425, 0.4411764705882353, 0.4375, 0.3723404255319149, 0.4230769230769231, 0.38823529411764707, 0.4177215189873418, 0.42857142857142855, 0.40789473684210525, 0.3953488372093023, 0.40476190476190477, 0.5094339622641509, 0.39759036144578314, 0.39285714285714285] 
Recall: [0.5333333333333333, 0.7555555555555555, 0.6888888888888889, 0.6666666666666666, 0.7111111111111111, 0.8, 0.7555555555555555, 0.6666666666666666, 0.6222222222222222, 0.7777777777777778, 0.7333333333333333, 0.7333333333333333, 0.7333333333333333, 0.6666666666666666, 0.6888888888888889, 0.7555555555555555, 0.7555555555555555, 0.6, 0.7333333333333333, 0.7333333333333333] 
F1: [0.4571428571428572, 0.5439999999999999, 0.5166666666666667, 0.5263157894736841, 0.5517241379310346, 0.5333333333333333, 0.5439999999999999, 0.5309734513274336, 0.5137614678899083, 0.5035971223021583, 0.5365853658536585, 0.5076923076923077, 0.5322580645161289, 0.

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: P

Precision: [0.6666666666666666, 0.0, 0.16666666666666666, 0.5, 0.3, 0.55, 0.0, 0.0, 0.0, 0.5555555555555556, 0.4, 0.7142857142857143, 0.0, 0.0, 0.0, 0.0, 0.0, 0.42857142857142855, 0.0, 0.045454545454545456] 
Recall: [0.20408163265306123, 0.0, 0.04081632653061224, 0.32653061224489793, 0.061224489795918366, 0.22448979591836735, 0.0, 0.0, 0.0, 0.30612244897959184, 0.08163265306122448, 0.20408163265306123, 0.0, 0.0, 0.0, 0.0, 0.0, 0.061224489795918366, 0.0, 0.04081632653061224] 
F1: [0.3125, 0.0, 0.06557377049180327, 0.39506172839506176, 0.10169491525423728, 0.3188405797101449, 0.0, 0.0, 0.0, 0.3947368421052632, 0.13559322033898302, 0.31746031746031744, 0.0, 0.0, 0.0, 0.0, 0.0, 0.10714285714285714, 0.0, 0.04301075268817204] 
Runtime: [10.247355699539185, 10.600005626678467, 10.675640344619751, 10.954608678817749, 11.290869235992432, 11.323673009872437, 11.458093404769897, 11.599946975708008, 12.08187460899353, 12.139231204986572, 12.296276569366455, 12.524097919464111, 12.870131254196167, 

KeyboardInterrupt: ignored

In [0]:
for i, config in enumerate(get_experimental_config()):
    print('Experiment', i + 23, '\b:', config)
    run_20_experiments(**config)
    print()

Experiment 23 : {'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 7, 'oversample': False, 'oversampler': None}


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: P

Precision: [0.0, 0.5555555555555556, 0.5121951219512195, 0.5, 0.42857142857142855, 0.0, 0.5454545454545454, 0.5, 0.0, 0.0, 0.5, 0.0, 0.25, 0.47619047619047616, 0.4482758620689655, 0.5384615384615384, 0.6, 0.5714285714285714, 0.65, 0.0] 
Recall: [0.0, 0.10869565217391304, 0.45652173913043476, 0.08695652173913043, 0.13043478260869565, 0.0, 0.2608695652173913, 0.2826086956521739, 0.0, 0.0, 0.13043478260869565, 0.0, 0.08695652173913043, 0.43478260869565216, 0.2826086956521739, 0.15217391304347827, 0.32608695652173914, 0.34782608695652173, 0.2826086956521739, 0.0] 
F1: [0.0, 0.1818181818181818, 0.48275862068965514, 0.14814814814814817, 0.2, 0.0, 0.3529411764705882, 0.3611111111111111, 0.0, 0.0, 0.20689655172413793, 0.0, 0.12903225806451613, 0.4545454545454545, 0.3466666666666666, 0.23728813559322035, 0.4225352112676057, 0.4324324324324324, 0.3939393939393939, 0.0] 
Runtime: [18.33475112915039, 18.63843321800232, 18.87383198738098, 19.155840635299683, 19.162107229232788, 19.479313373565674, 

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Epoch 1 LR = 0.3837283875664912
Epoch 2 LR = 18.475299227278384
Epoch 3 LR = 0.8116860384445012
Epoch 4 LR = 0.6924535463219332
Epoch 5 LR = 0.7507929384754208
Epoch 6 LR = 0.8206893427789904
Epoch 7 LR = 0.9780560783796599
Epoch 8 LR = 1.8153122871482588
Epoch 9 LR = 0.4789339707085848
Epoch 10 LR = 0.5084904106071607
Epoch 11 LR = 0.5314775290569727
Epoch 12 LR = 0.564147564065248
Epoch 13 LR = 0.6155809204586641
Epoch 14 LR = 0.6771322894509741
Epoch 15 LR = 0.760283452426409
Epoch 16 LR = 0.9101446979613824
Epoch 17 LR = 1.2293554910016469
Epoch 18 LR = 2.2600087990626547
Epoch 19 LR = 7.736659080900239
Epoch 20 LR = 7.898799493978638
Epoch 21 LR = 2.4262541118852163
Epoch 22 LR = 6.392888769663871
Epoch 23 LR = 4.3366087313676465
Epoch 24 LR = 5.736976176826669
Epoch 25 LR = 15.290396916805937
Epoch 26 LR = 15.58511810844416
Epoch 27 LR = 5.907356539517416
Epoch 28 LR = 4.718051340804164
Epoch 29 LR = 3.718742391022823
Epoch 30 LR = 5.900572570094115
Epoch 31 LR = 6.67853490904708

...

Epoch 1 LR = 1.4085297353615163
Epoch 2 LR = 19.37016570200252
Epoch 3 LR = 6.229632349855986
Epoch 4 LR = 17.426356492524796
Epoch 5 LR = 57.06963106082768
Epoch 6 LR = 1.2475214409642637
Epoch 7 LR = 1.2476952108069823
Epoch 8 LR = 1.247781417868116
Epoch 9 LR = 1.2478457329387167
Epoch 10 LR = 1.2479041141181235
Epoch 11 LR = 1.247945977436249
Epoch 12 LR = 1.2479736403272763
Epoch 13 LR = 1.248023213810694
Epoch 14 LR = 1.2480485590792783
Epoch 15 LR = 1.2480800332083062
Epoch 16 LR = 1.2480915462656081
Epoch 17 LR = 1.2480975814208222
Epoch 18 LR = 1.2480999954992502
Epoch 19 LR = 1.2480999026499073
Epoch 20 LR = 1.2481055664851108
Epoch 21 LR = 1.2481068663889676
Epoch 22 LR = 1.2481200512811013
Epoch 23 LR = 1.2481255295928486
Epoch 24 LR = 1.248132029347129
Epoch 25 LR = 1.248143543363744
Epoch 26 LR = 1.2481507861604573
Epoch 27 LR = 1.2481496718785687
Epoch 28 LR = 1.2481548718777373
Epoch 29 LR = 1.248157936183251
Epoch 30 LR = 1.2481473504643574
Epoch 31 LR = 1.248144564778

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Epoch 1 LR = 6.103936582745441
Epoch 2 LR = 3.8432469325949166
Epoch 3 LR = 9.046234779715197
Epoch 4 LR = 11.990779932944479
Epoch 5 LR = 5.932860558884748
Epoch 6 LR = 77.12966588646887
Epoch 7 LR = 0.6608211707093734
Epoch 8 LR = 0.6608211707093734
Epoch 9 LR = 0.6608211707093734
Epoch 10 LR = 0.6608211707093734
Epoch 11 LR = 0.6608211707093734
Epoch 12 LR = 0.6608211707093734
Epoch 13 LR = 0.6608211707093734
Epoch 14 LR = 0.6608211707093734
Epoch 15 LR = 0.6608211707093734
Epoch 16 LR = 0.6608211707093734
Epoch 17 LR = 0.6608211707093734
Epoch 18 LR = 0.6608211707093734
Epoch 19 LR = 0.6608211707093734
Epoch 20 LR = 0.6608211707093734
Epoch 21 LR = 0.6608211707093734
Epoch 22 LR = 0.6608211707093734
Epoch 23 LR = 0.6608211707093734
Epoch 24 LR = 0.6608211707093734
Epoch 25 LR = 0.6608211707093734
Epoch 26 LR = 0.6608211707093734
Epoch 27 LR = 0.6608211707093734
Epoch 28 LR = 0.6608211707093734
Epoch 29 LR = 0.6608211707093734
Epoch 30 LR = 0.6608211707093734
Epoch 31 LR = 0.6608211

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Epoch 1 LR = 2.6730254228896584
Epoch 2 LR = 11.574091739207896
Epoch 3 LR = 11.733058350967749
Epoch 4 LR = 11.288783181968391
Epoch 5 LR = 8.218885950402697
Epoch 6 LR = 130.1664754311104
Epoch 7 LR = 0.42340836391986353
Epoch 8 LR = 0.42340836391986353
Epoch 9 LR = 0.42340836391986353
Epoch 10 LR = 0.42340836391986353
Epoch 11 LR = 0.42340836391986353
Epoch 12 LR = 0.42340836391986353
Epoch 13 LR = 0.42340836391986353
Epoch 14 LR = 0.42340836391986353
Epoch 15 LR = 0.42340836391986353
Epoch 16 LR = 0.42340836391986353
Epoch 17 LR = 0.42340836391986353
Epoch 18 LR = 0.42340836391986353
Epoch 19 LR = 0.42340836391986353
Epoch 20 LR = 0.42340836391986353
Epoch 21 LR = 0.42340836391986353
Epoch 22 LR = 0.42340836391986353
Epoch 23 LR = 0.42340836391986353
Epoch 24 LR = 0.42340836391986353
Epoch 25 LR = 0.42340836391986353
Epoch 26 LR = 0.42340836391986353
Epoch 27 LR = 0.42340836391986353
Epoch 28 LR = 0.42340836391986353
Epoch 29 LR = 0.42340836391986353
Epoch 30 LR = 0.423408363919863

In [0]:
# Add 36 to the numbers below, since it should've been + 37
for i, config in enumerate(get_experimental_config()):
    print('Experiment', i + 1, '\b:', config)
    run_20_experiments(**config)
    print()

Experiment 1 : {'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 3, 'oversample': True, 'oversampler': 'random'}
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
















/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: P

Precision: [0.0, 0.2192513368983957, 0.2192513368983957, 0.2192513368983957, 0.2192513368983957, 0.0, 0.2192513368983957, 0.0, 0.0, 0.2192513368983957, 0.0, 0.0, 0.2192513368983957, 0.2192513368983957, 0.2192513368983957, 0.6428571428571429, 0.0, 0.2192513368983957, 0.5714285714285714, 0.0] 
Recall: [0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.6585365853658537, 0.0, 1.0, 0.3902439024390244, 0.0] 
F1: [0.0, 0.35964912280701755, 0.35964912280701755, 0.35964912280701755, 0.35964912280701755, 0.0, 0.35964912280701755, 0.0, 0.0, 0.35964912280701755, 0.0, 0.0, 0.35964912280701755, 0.35964912280701755, 0.35964912280701755, 0.6506024096385543, 0.0, 0.35964912280701755, 0.463768115942029, 0.0] 
Runtime: [6.100682973861694, 6.362764358520508, 6.5914552211761475, 6.668995141983032, 6.931625604629517, 6.836276531219482, 7.402844667434692, 7.347950220108032, 7.570419073104858, 7.5557332038879395, 7.931770086288452, 8.02157473564148, 8.07624101638794, 8.2227559089660

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: P

Precision: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 
Recall: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 
F1: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 
Runtime: [8.486796855926514, 8.810489416122437, 9.07443642616272, 9.184412240982056, 9.367955923080444, 9.518422365188599, 10.169541358947754, 10.058487176895142, 10.291836738586426, 10.429871797561646, 10.515719890594482, 10.709787368774414, 10.846442222595215, 10.98765778541565, 11.25841498374939, 11.331040859222412, 11.637286901473999, 11.681021451950073, 12.013882637023926, 12.157766819000244]

Experiment 3 : {'deep_learner': True, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 4, 'oversample': True, 'oversampler': 'random'}


...

Precision: [0.6521739130434783, 0.35294117647058826, 0.6842105263157895, 0.6578947368421053, 0.0, 0.75, 0.5517241379310345, 0.72, 0.7352941176470589, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.6153846153846154, 0.6944444444444444, 0.0, 0.75] 
Recall: [0.3191489361702128, 0.1276595744680851, 0.2765957446808511, 0.5319148936170213, 0.0, 0.1276595744680851, 0.3404255319148936, 0.3829787234042553, 0.5319148936170213, 0.0, 0.0, 0.0, 0.0, 0.02127659574468085, 0.0, 0.0, 0.1702127659574468, 0.5319148936170213, 0.0, 0.5106382978723404] 
F1: [0.4285714285714286, 0.18749999999999997, 0.393939393939394, 0.5882352941176471, 0.0, 0.2181818181818182, 0.4210526315789473, 0.4999999999999999, 0.617283950617284, 0.0, 0.0, 0.0, 0.0, 0.04166666666666667, 0.0, 0.0, 0.26666666666666666, 0.6024096385542169, 0.0, 0.6075949367088608] 
Runtime: [97.55601501464844, 87.71646213531494, 86.32399773597717, 87.23815822601318, 87.37021517753601, 86.94201183319092, 86.96414613723755, 87.18497729301453, 87.88393664360046, 88.0

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: P

Precision: [0.0, 0.0, 0.5116279069767442, 0.625, 0.0, 0.6875, 0.0, 0.6785714285714286, 0.6842105263157895, 0.5555555555555556, 0.6956521739130435, 0.4, 0.6, 0.4864864864864865, 0.4722222222222222, 0.0, 0.44680851063829785, 0.42857142857142855, 0.0, 0.0] 
Recall: [0.0, 0.0, 0.55, 0.25, 0.0, 0.275, 0.0, 0.475, 0.325, 0.125, 0.4, 0.2, 0.225, 0.45, 0.425, 0.0, 0.525, 0.45, 0.0, 0.0] 
F1: [0.0, 0.0, 0.5301204819277109, 0.35714285714285715, 0.0, 0.3928571428571429, 0.0, 0.5588235294117648, 0.4406779661016949, 0.20408163265306123, 0.507936507936508, 0.26666666666666666, 0.3272727272727273, 0.4675324675324675, 0.4473684210526316, 0.0, 0.48275862068965514, 0.4390243902439024, 0.0, 0.0] 
Runtime: [95.06877946853638, 95.40501499176025, 95.04670691490173, 95.43181324005127, 95.71119689941406, 95.84819340705872, 96.19909715652466, 97.25687718391418, 96.96993279457092, 97.15808129310608, 97.30950713157654, 97.80021357536316, 97.62085771560669, 97.92307567596436, 98.11983942985535, 98.38781380653381,

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: P

Precision: [0.0, 0.0, 0.25, 0.0, 0.0, 0.5714285714285714, 0.0, 0.0, 0.4166666666666667, 0.0, 0.375, 0.0, 0.0, 0.0, 0.6875, 0.72, 0.6129032258064516, 0.4375, 0.0, 0.0] 
Recall: [0.0, 0.0, 0.18421052631578946, 0.0, 0.0, 0.3157894736842105, 0.0, 0.0, 0.39473684210526316, 0.0, 0.07894736842105263, 0.0, 0.0, 0.0, 0.5789473684210527, 0.47368421052631576, 0.5, 0.3684210526315789, 0.0, 0.0] 
F1: [0.0, 0.0, 0.2121212121212121, 0.0, 0.0, 0.4067796610169492, 0.0, 0.0, 0.40540540540540543, 0.0, 0.13043478260869565, 0.0, 0.0, 0.0, 0.6285714285714286, 0.5714285714285714, 0.5507246376811595, 0.39999999999999997, 0.0, 0.0] 
Runtime: [106.04757261276245, 105.65199661254883, 105.49215316772461, 106.071533203125, 105.89570784568787, 106.355642080307, 107.05273246765137, 107.6436767578125, 108.12719011306763, 108.88911747932434, 109.44799828529358, 109.65938806533813, 108.96097183227539, 108.42128610610962, 108.92221426963806, 109.5438003540039, 109.8854169845581, 114.13182234764099, 110.41654324531555, 1

In [0]:
for i, config in enumerate(get_experimental_config()):
    print('Experiment', i + 65, '\b:', config)
    run_20_experiments(**config)
    print()

Experiment 65 : {'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 7, 'oversample': False, 'oversampler': None}
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
















/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: P

Precision: [0.0, 0.0, 0.6666666666666666, 0.0, 0.0, 0.0, 0.625, 0.5833333333333334, 0.0, 0.0, 0.6666666666666666, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6666666666666666, 0.6428571428571429, 0.0] 
Recall: [0.0, 0.0, 0.4878048780487805, 0.0, 0.0, 0.0, 0.36585365853658536, 0.17073170731707318, 0.0, 0.0, 0.0975609756097561, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.34146341463414637, 0.43902439024390244, 0.0] 
F1: [0.0, 0.0, 0.5633802816901409, 0.0, 0.0, 0.0, 0.4615384615384615, 0.2641509433962264, 0.0, 0.0, 0.1702127659574468, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4516129032258065, 0.5217391304347826, 0.0] 
Runtime: [5.774722337722778, 5.670209169387817, 5.7578606605529785, 6.085271835327148, 6.54269003868103, 6.134514570236206, 6.318349361419678, 6.747880220413208, 6.902298212051392, 7.019473314285278, 7.2914533615112305, 7.481217384338379, 7.661841154098511, 7.576342821121216, 7.931863784790039, 8.137334108352661, 8.599123001098633, 8.81286883354187, 8.855093002319336, 9.233856439590454]

Experiment 66 : {'

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: [0.42105263157894735, 0.38271604938271603, 0.375, 0.4025974025974026, 0.3950617283950617, 0.3595505617977528, 0.36666666666666664, 0.38095238095238093, 0.47619047619047616, 0.4, 0.5370370370370371, 0.42857142857142855, 0.42424242424242425, 0.45454545454545453, 0.26881720430107525, 0.40963855421686746, 0.367816091954023, 0.52, 0.5, 0.4225352112676056] 
Recall: [0.8648648648648649, 0.8378378378378378, 0.7297297297297297, 0.8378378378378378, 0.8648648648648649, 0.8648648648648649, 0.8918918918918919, 0.8648648648648649, 0.8108108108108109, 0.8648648648648649, 0.7837837837837838, 0.8108108108108109, 0.7567567567567568, 0.8108108108108109, 0.6756756756756757, 0.918918918918919, 0.8648648648648649, 0.7027027027027027, 0.7027027027027027, 0.8108108108108109] 
F1: [0.5663716814159292, 0.5254237288135593, 0.49541284403669716, 0.543859649122807, 0.5423728813559322, 0.5079365079365079, 0.5196850393700787, 0.5289256198347108, 0.6, 0.5470085470085471, 0.6373626373626374, 0.56074766355140

...

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: P

Precision: [0.45714285714285713, 0.8, 0.8333333333333334, 0.38636363636363635, 0.21390374331550802, 0.8, 0.0, 1.0, 0.0, 0.21390374331550802, 0.21390374331550802, 0.3888888888888889, 0.21390374331550802, 0.7083333333333334, 0.0, 0.0, 0.21390374331550802, 0.47058823529411764, 1.0, 0.0] 
Recall: [0.8, 0.1, 0.125, 0.85, 1.0, 0.2, 0.0, 0.125, 0.0, 1.0, 1.0, 0.875, 1.0, 0.425, 0.0, 0.0, 1.0, 0.8, 0.075, 0.0] 
F1: [0.5818181818181818, 0.1777777777777778, 0.21739130434782608, 0.53125, 0.3524229074889868, 0.32000000000000006, 0.0, 0.2222222222222222, 0.0, 0.3524229074889868, 0.3524229074889868, 0.5384615384615385, 0.3524229074889868, 0.53125, 0.0, 0.0, 0.3524229074889868, 0.5925925925925927, 0.13953488372093023, 0.0] 
Runtime: [31.35574507713318, 31.563060760498047, 31.884485006332397, 32.55603504180908, 32.64806079864502, 33.11181569099426, 33.47386908531189, 33.702414751052856, 34.877206325531006, 34.61296725273132, 34.88226389884949, 35.3148353099823, 35.64367198944092, 35.86411643028259, 36

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Retrying...


NameError: ignored

In [0]:
for i, config in enumerate(get_experimental_config()):
    print('Experiment', i + 69, '\b:', config)
    run_20_experiments(**config)
    print()

Experiment 69 : {'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 2, 'k': 3, 'oversample': False, 'oversampler': None}


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: P

Precision: [0.0, 0.0, 0.0, 0.21390374331550802, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 
Recall: [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.025, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 
F1: [0.0, 0.0, 0.0, 0.3524229074889868, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.04878048780487806, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 
Runtime: [21.02871799468994, 21.360553979873657, 21.295980215072632, 21.46914553642273, 22.100239515304565, 22.16685152053833, 22.600401878356934, 22.898377418518066, 22.948622465133667, 23.501821279525757, 23.854347467422485, 24.079328298568726, 24.37704849243164, 24.64214038848877, 25.035184860229492, 25.393364191055298, 25.599226236343384, 26.073599815368652, 26.394291400909424, 26.982887268066406]

Experiment 70 : {'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 2, 'k': 3, 'oversample': True, 'oversampler': 'random'}


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: P

Retrying...


...

Precision: [0.0, 0.0, 0.0, 0.0, 0.0, 0.2111801242236025, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 
Recall: [0.0, 0.0, 0.0, 0.0, 0.0, 0.7391304347826086, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 
F1: [0.0, 0.0, 0.0, 0.0, 0.0, 0.32850241545893727, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 
Runtime: [124.02261543273926, 123.92290878295898, 123.78251576423645, 124.38246059417725, 126.44878888130188, 130.52380752563477, 129.96092414855957, 130.10972547531128, 130.92155647277832, 130.61631321907043, 130.7309606075287, 131.15006732940674, 131.30131888389587, 132.06566047668457, 131.78257775306702, 131.88934588432312, 132.34483408927917, 132.25442242622375, 132.51087856292725, 133.6423943042755]

Experiment 88 : {'deep_learner': True, 'n_layers': 2, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 7, 'oversample': True, 'oversampler': 'random'}


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: P

Retrying...


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: [0.22459893048128343, 0.15873015873015872, 0.22459893048128343, 0.22459893048128343, 0.22459893048128343, 0.0, 0.0, 0.2597402597402597, 0.22459893048128343, 0.22459893048128343, 0.22459893048128343, 0.0, 0.22459893048128343, 0.0, 0.75, 0.22459893048128343, 0.22459893048128343, 0.22459893048128343, 0.0, 0.22459893048128343] 
Recall: [1.0, 0.23809523809523808, 1.0, 1.0, 1.0, 0.0, 0.0, 0.9523809523809523, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.07142857142857142, 1.0, 1.0, 1.0, 0.0, 1.0] 
F1: [0.36681222707423583, 0.19047619047619047, 0.36681222707423583, 0.36681222707423583, 0.36681222707423583, 0.0, 0.0, 0.40816326530612246, 0.36681222707423583, 0.36681222707423583, 0.36681222707423583, 0.0, 0.36681222707423583, 0.0, 0.13043478260869565, 0.36681222707423583, 0.36681222707423583, 0.36681222707423583, 0.0, 0.36681222707423583] 
Runtime: [137.0131139755249, 138.31368398666382, 138.7264564037323, 138.92400240898132, 139.44460582733154, 139.000563621521, 139.678941488266, 139.744644880294

In [0]:
# List the remaining configs
for i, config in enumerate(get_experimental_config()):
    print(config)

{'deep_learner': True, 'n_layers': 3, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 2, 'k': 7, 'oversample': False, 'oversampler': None}
{'deep_learner': True, 'n_layers': 3, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 2, 'k': 7, 'oversample': True, 'oversampler': 'random'}
{'deep_learner': True, 'n_layers': 3, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 3, 'oversample': False, 'oversampler': None}
{'deep_learner': True, 'n_layers': 3, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 3, 'oversample': True, 'oversampler': 'random'}
{'deep_learner': True, 'n_layers': 3, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 4, 'oversample': False, 'oversampler': None}
{'deep_learner': True, 'n_layers': 3, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 4, 'oversampl

In [0]:
for i, config in enumerate(get_experimental_config()):
    print('Experiment', i + 89, '\b:', config)
    run_20_experiments(**config)
    print()

Experiment 89 : {'deep_learner': False, 'n_layers': 1, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': False, 'embedding_dims': 2, 'k': 3, 'oversample': False, 'oversampler': None}
Precision: [0.6923076923076923, 0.6923076923076923, 0.6923076923076923, 0.6923076923076923, 0.6923076923076923, 0.6923076923076923, 0.6923076923076923, 0.6923076923076923, 0.6923076923076923, 0.6923076923076923, 0.6923076923076923, 0.6923076923076923, 0.6923076923076923, 0.6923076923076923, 0.6923076923076923, 0.6923076923076923, 0.6923076923076923, 0.6923076923076923, 0.6923076923076923, 0.6923076923076923] 
Recall: [0.4909090909090909, 0.4909090909090909, 0.4909090909090909, 0.4909090909090909, 0.4909090909090909, 0.4909090909090909, 0.4909090909090909, 0.4909090909090909, 0.4909090909090909, 0.4909090909090909, 0.4909090909090909, 0.4909090909090909, 0.4909090909090909, 0.4909090909090909, 0.4909090909090909, 0.4909090909090909, 0.4909090909090909, 0.4909090909090909, 0.4909090909090909, 0.49090

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: P

Precision: [0.0, 0.0, 0.6, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.6190476190476191, 0.4782608695652174, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5652173913043478, 0.0, 0.0, 0.56] 
Recall: [0.0, 0.0, 0.06976744186046512, 0.0, 0.0, 0.023255813953488372, 0.0, 0.0, 0.0, 0.3023255813953488, 0.2558139534883721, 0.0, 0.0, 0.0, 0.0, 0.11627906976744186, 0.3023255813953488, 0.0, 0.0, 0.32558139534883723] 
F1: [0.0, 0.0, 0.125, 0.0, 0.0, 0.04444444444444444, 0.0, 0.0, 0.0, 0.40625, 0.33333333333333337, 0.0, 0.0, 0.0, 0.0, 0.18867924528301888, 0.39393939393939387, 0.0, 0.0, 0.411764705882353] 
Runtime: [24.56185269355774, 24.903547525405884, 25.723531246185303, 26.002830982208252, 26.484416246414185, 26.588226318359375, 27.91878366470337, 27.487848043441772, 27.828033685684204, 28.28235626220703, 28.67511487007141, 29.242738962173462, 29.206420421600342, 29.62108302116394, 30.117681741714478, 30.48550033569336, 30.54252290725708, 30.88552451133728, 31.254393815994263, 31.74796175956726]

Experiment 114 : {'deep_lear

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: P

Precision: [0.5588235294117647, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5666666666666667, 0.0, 0.5423728813559322, 0.0, 0.4383561643835616, 0.0, 0.0, 0.0, 0.0, 0.625] 
Recall: [0.3958333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3541666666666667, 0.0, 0.6666666666666666, 0.0, 0.6666666666666666, 0.0, 0.0, 0.0, 0.0, 0.10416666666666667] 
F1: [0.4634146341463415, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4358974358974359, 0.0, 0.5981308411214954, 0.0, 0.5289256198347108, 0.0, 0.0, 0.0, 0.0, 0.1785714285714286] 
Runtime: [38.3620388507843, 38.45114064216614, 38.92091965675354, 39.63864779472351, 39.70532298088074, 40.01567840576172, 42.032859325408936, 41.12499666213989, 41.178709983825684, 41.893688440322876, 42.12881255149841, 42.27216124534607, 42.55722141265869, 42.643882513046265, 43.50141787528992, 43.53487992286682, 43.98196339607239, 44.201452016830444, 44.600630044937134, 44.82410907745361]

Experiment 116 : {'deep_learner': True, 'n_layers': 3, 'batch

...

Precision: [0.5681818181818182, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7142857142857143, 0.0, 0.7727272727272727, 0.5128205128205128, 0.0, 0.0, 0.6756756756756757, 0.5370370370370371, 0.5909090909090909, 0.5789473684210527, 0.0, 0.0] 
Recall: [0.5555555555555556, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4444444444444444, 0.0, 0.37777777777777777, 0.4444444444444444, 0.0, 0.0, 0.5555555555555556, 0.6444444444444445, 0.5777777777777777, 0.4888888888888889, 0.0, 0.0] 
F1: [0.5617977528089888, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5479452054794521, 0.0, 0.5074626865671642, 0.47619047619047616, 0.0, 0.0, 0.6097560975609757, 0.585858585858586, 0.5842696629213482, 0.5301204819277109, 0.0, 0.0] 
Runtime: [115.16277885437012, 115.55167365074158, 116.72530579566956, 116.59382605552673, 117.62109613418579, 117.52585625648499, 118.56766319274902, 118.61342525482178, 119.75069665908813, 119.80987453460693, 119.76387333869934, 120.0166084766388, 120.4289903640747, 120.33278751373291, 120.64875841140747, 12

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: [0.45454545454545453, 0.41025641025641024, 0.4406779661016949, 0.38636363636363635, 0.3875, 0.41935483870967744, 0.4050632911392405, 0.4264705882352941, 0.4142857142857143, 0.41025641025641024, 0.3564356435643564, 0.4189189189189189, 0.3394495412844037, 0.43548387096774194, 0.36363636363636365, 0.3950617283950617, 0.4142857142857143, 0.45161290322580644, 0.3595505617977528, 0.4305555555555556] 
Recall: [0.7142857142857143, 0.7619047619047619, 0.6190476190476191, 0.8095238095238095, 0.7380952380952381, 0.6190476190476191, 0.7619047619047619, 0.6904761904761905, 0.6904761904761905, 0.7619047619047619, 0.8571428571428571, 0.7380952380952381, 0.8809523809523809, 0.6428571428571429, 0.38095238095238093, 0.7619047619047619, 0.6904761904761905, 0.6666666666666666, 0.7619047619047619, 0.7380952380952381] 
F1: [0.5555555555555556, 0.5333333333333333, 0.5148514851485149, 0.523076923076923, 0.5081967213114755, 0.5, 0.5289256198347108, 0.5272727272727272, 0.5178571428571428, 0.533333333

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: P

Precision: [0.0, 0.7142857142857143, 0.0, 0.0, 0.5, 0.5, 0.6666666666666666, 0.6571428571428571, 0.0, 0.6818181818181818, 0.0, 0.6571428571428571, 0.0, 0.6153846153846154, 0.75, 0.43478260869565216, 0.6388888888888888, 0.5, 0.0, 0.0] 
Recall: [0.0, 0.39473684210526316, 0.0, 0.0, 0.23684210526315788, 0.39473684210526316, 0.5789473684210527, 0.6052631578947368, 0.0, 0.39473684210526316, 0.0, 0.6052631578947368, 0.0, 0.631578947368421, 0.47368421052631576, 0.2631578947368421, 0.6052631578947368, 0.47368421052631576, 0.0, 0.0] 
F1: [0.0, 0.5084745762711864, 0.0, 0.0, 0.32142857142857145, 0.4411764705882353, 0.619718309859155, 0.6301369863013698, 0.0, 0.5, 0.0, 0.6301369863013698, 0.0, 0.6233766233766234, 0.5806451612903226, 0.32786885245901637, 0.6216216216216216, 0.4864864864864865, 0.0, 0.0] 
Runtime: [126.45621395111084, 126.9210433959961, 127.54411458969116, 127.79214763641357, 128.2448604106903, 128.8437385559082, 129.19644856452942, 130.04548859596252, 128.75750184059143, 130.4243388

In [0]:
for i, config in enumerate(get_experimental_config()):
    print('Experiment', i + 128, '\b:', config)
    run_20_experiments(**config)
    print()

Experiment 128 : {'deep_learner': True, 'n_layers': 3, 'batchnorm': True, 'lipschitz_lr': False, 'reduce_data': True, 'embedding_dims': 3, 'k': 5, 'oversample': True, 'oversampler': 'random'}
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where














Precision: [0.5147058823529411, 0.5409836065573771, 0.5, 0.4827586206896552, 0.5178571428571429, 0.5185185185185185, 0.4461538461538462, 0.5, 0.4827586206896552, 0.47368421052631576, 0.4897959183673469, 0.4270833333333333, 0.40476190476190477, 0.4528301886792453, 0.5, 0.5, 0.36666666666666664, 0.4915254237288136, 0.47619047619047616, 0.4745762711864407] 
Recall: [0.7777777777777778, 0.7333333333333333, 0.6666666666666666, 0.6222222222222222, 0.6444444444444445, 0.6222222222222222, 0.6444444444444445, 0.5555555555555556, 0.6222222222222222, 0.8, 0.5333333333333333, 0.9111111111111111, 0.7555555555555555, 0.53333

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: P

Precision: [0.7, 0.9166666666666666, 0.7368421052631579, 0.6, 0.45454545454545453, 0.5, 0.7391304347826086, 0.6875, 0.7142857142857143, 0.5217391304347826, 0.0, 0.5142857142857142, 0.0, 0.0, 0.7142857142857143, 0.0, 0.8333333333333334, 0.75, 0.8, 0.0] 
Recall: [0.35, 0.275, 0.35, 0.375, 0.125, 0.125, 0.425, 0.275, 0.375, 0.3, 0.0, 0.45, 0.0, 0.0, 0.125, 0.0, 0.25, 0.075, 0.2, 0.0] 
F1: [0.4666666666666667, 0.4230769230769231, 0.47457627118644063, 0.4615384615384615, 0.196078431372549, 0.2, 0.5396825396825397, 0.3928571428571429, 0.4918032786885245, 0.38095238095238093, 0.0, 0.48, 0.0, 0.0, 0.2127659574468085, 0.0, 0.3846153846153846, 0.13636363636363635, 0.32000000000000006, 0.0] 
Runtime: [13.647724151611328, 14.22074031829834, 14.368570566177368, 15.341716766357422, 15.182010412216187, 15.679547786712646, 15.761054515838623, 16.209338903427124, 16.47408890724182, 16.765440940856934, 17.03448486328125, 17.276347160339355, 18.322862148284912, 18.01666021347046, 18.424009799957275, 18.5

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: [0.3142857142857143, 0.3787878787878788, 0.3611111111111111, 0.3783783783783784, 0.38461538461538464, 0.42105263157894735, 0.43478260869565216, 0.39344262295081966, 0.36486486486486486, 0.4, 0.39705882352941174, 0.39622641509433965, 0.4166666666666667, 0.41509433962264153, 0.42592592592592593, 0.39285714285714285, 0.34444444444444444, 0.4166666666666667, 0.4339622641509434, 0.3561643835616438] 
Recall: [0.9428571428571428, 0.7142857142857143, 0.7428571428571429, 0.8, 0.7142857142857143, 0.6857142857142857, 0.5714285714285714, 0.6857142857142857, 0.7714285714285715, 0.6857142857142857, 0.7714285714285715, 0.6, 0.7142857142857143, 0.6285714285714286, 0.6571428571428571, 0.6285714285714286, 0.8857142857142857, 0.5714285714285714, 0.6571428571428571, 0.7428571428571429] 
F1: [0.4714285714285714, 0.49504950495049505, 0.4859813084112149, 0.5137614678899083, 0.5, 0.5217391304347826, 0.49382716049382713, 0.5, 0.4954128440366973, 0.5052631578947367, 0.5242718446601942, 0.477272727272

...

Retrying...


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: P

Retrying...


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: P

KeyboardInterrupt: ignored

In [0]:
for i, config in enumerate(get_experimental_config()):
    print('Experiment', i + 143, '\b:', config)
    run_20_experiments(**config)
    print()

Experiment 143 : {'deep_learner': True, 'n_layers': 3, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 2, 'k': 7, 'oversample': False, 'oversampler': None}


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: P

Retrying...


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: P

Retrying...


...

Retrying...


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: P

Retrying...


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Retrying...


In [0]:
for i, config in enumerate(get_experimental_config()):
    print('Experiment', i + 144, '\b:', config)
    run_20_experiments(**config)
    print()

Experiment 144 : {'deep_learner': True, 'n_layers': 3, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 2, 'k': 7, 'oversample': True, 'oversampler': 'random'}
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
















/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: P

Retrying...


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Retrying...
Precision: [0.0, 0.20855614973262032, 0.0, 0.0, 0.20855614973262032, 0.0, 0.0, 0.20855614973262032, 0.3645833333333333, 0.5, 0.20855614973262032, 0.0, 0.20855614973262032, 0.0, 0.23214285714285715, 0.20855614973262032, 0.20855614973262032, 0.0, 0.20855614973262032, 0.0] 
Recall: [0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.8974358974358975, 0.10256410256410256, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0] 
F1: [0.0, 0.3451327433628319, 0.0, 0.0, 0.3451327433628319, 0.0, 0.0, 0.3451327433628319, 0.5185185185185185, 0.1702127659574468, 0.3451327433628319, 0.0, 0.3451327433628319, 0.0, 0.37681159420289856, 0.3451327433628319, 0.3451327433628319, 0.0, 0.3451327433628319, 0.0] 
Runtime: [9.692952394485474, 9.427556037902832, 9.827488899230957, 10.009650945663452, 10.593467950820923, 10.651135444641113, 10.50174355506897, 11.095602035522461, 11.989895105361938, 12.098702669143677, 12.527217149734497, 13.037216663360596, 13.377411842346191, 13.669731140136719, 14.75746655464172

...

Precision: [0.24725274725274726, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 
Recall: [0.9782608695652174, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 
F1: [0.3947368421052632, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 
Runtime: [80.66674041748047, 80.92930340766907, 84.03194332122803, 82.99212265014648, 83.9893445968628, 84.43142247200012, 84.34698295593262, 84.5459406375885, 84.8350760936737, 85.3968255519867, 86.07058334350586, 86.09104490280151, 86.94416999816895, 87.29125380516052, 87.88001537322998, 88.70695734024048, 88.99481749534607, 89.40448474884033, 89.20756363868713, 89.79266929626465]

Experiment 154 : {'deep_learner': True, 'n_layers': 3, 'batchnorm': True, 'lipschitz_lr': True, 'reduce_data': True, 'embedding_dims': 3, 'k': 7, 'oversample': True, 'oversampler': 'random'}


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Retrying...


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: P

KeyboardInterrupt: ignored